In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_regression
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from folium import Map, Circle
from random import randint

In [2]:
boston_df = pd.read_csv('boston_corrected_corrected.csv')
boston_gdf  = gpd.GeoDataFrame(boston_df, geometry=gpd.points_from_xy(boston_df.LAT, boston_df.LON))

hospital_df = pd.read_csv('massachusetts-hospitals.csv')
hospital_gdf  = gpd.GeoDataFrame(hospital_df, geometry=gpd.points_from_xy(hospital_df.SHAPE_Y, hospital_df.SHAPE_X))

school_df = pd.read_csv('massachusetts-schools-pre-k-through-high-school_new.csv')
school_gdf  = gpd.GeoDataFrame(school_df, geometry=gpd.points_from_xy(school_df.SHAPE_Y, school_df.SHAPE_X))

boston_gdf.set_crs(epsg=4326, inplace=True)
hospital_gdf.set_crs(epsg=4326, inplace=True)
school_gdf.set_crs(epsg=4326, inplace=True)

boston_gdf.to_crs(epsg=3857, inplace=True)
hospital_gdf.to_crs(epsg=3857, inplace=True)
school_gdf.to_crs(epsg=3857, inplace=True)

## Checking and cleaning the data

This data frame contains the following columns:

* TOWN a factor with levels given by town names
    * Should all be strings
    * All have capital letter at the beginning of each word

* TOWNNO a numeric vector corresponding to TOWN
    * Should be int
    * Should be same number as number of unique town names

* TRACT a numeric vector of tract ID numbers
    * Should be int
    * Should be >= 0

* LON a numeric vector of tract point longitudes in decimal degrees
    * Should be float
    * Should be between -180 and 180

* LAT a numeric vector of tract point latitudes in decimal degrees
    * Should be float
    * Should be between -90 and 90

* MEDV a numeric vector of median values of owner-occupied housing in USD 1000
    * Should be float

* CMEDV a numeric vector of corrected median values of owner-occupied housing in USD 1000
    * Should be float

* CRIM a numeric vector of per capita crime
    * Should be float

* ZN a numeric vector of proportions of residential land zoned for lots over 25000 sq. ft per town (constant for all Boston tracts)
    * Should be float

* INDUS a numeric vector of proportions of non-retail business acres per town (constant for all Boston tracts)
    * Should be float

* CHAS a factor with levels 1 if tract borders Charles River; 0 otherwise
    * Should be int

* NOX a numeric vector of nitric oxides concentration (parts per 10 million) per town
    * Should be float

* RM a numeric vector of average numbers of rooms per dwelling
    * Should be float

* AGE a numeric vector of proportions of owner-occupied units built prior to 1940
    * Should be float

* DIS a numeric vector of weighted distances to five Boston employment centres
    * Should be float

* RAD a numeric vector of an index of accessibility to radial highways per town (constant for all Boston tracts)
    * Should be float

* TAX a numeric vector full-value property-tax rate per USD 10,000 per town (constant for all Boston tracts)
    * Should be float

* PTRATIO a numeric vector of pupil-teacher ratios per town (constant for all Boston tracts)
    * Should be float

* B a numeric vector of 1000*(Bk - 0.63)^2 where Bk is the proportion of blacks
    * Should be float

* LSTAT a numeric vector of percentage values of lower status population
    * Should be float

Adapted from: https://nowosad.github.io/spData/reference/boston.html

### Checking that there are no null values and that dtypes are as expected

In [3]:
boston_gdf.head()

,Unnamed: 0,TOWN,TOWNNO,TRACT,LON,LAT,MEDV,CMEDV,CRIM,ZN,...,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,geometry
0,0,Nahant,0,2011,-70.927800,42.426000,24.0,24.0,0.00632,18.0,...,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,POINT (4722840.716 -11377630.320)
1,1,Swampscott,1,2021,-70.919764,42.481455,21.6,21.6,0.02731,0.0,...,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,POINT (4729013.939 -11374893.260)
2,2,Swampscott,1,2022,-70.897264,42.473777,34.7,34.7,0.02729,0.0,...,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,POINT (4728159.185 -11367235.395)
3,3,Marblehead,2,2031,-70.884407,42.490840,33.4,33.4,0.03237,0.0,...,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,POINT (4730058.638 -11362863.369)
4,4,Marblehead,2,2032,-70.874764,42.499371,36.2,36.2,0.06905,0.0,...,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,POINT (4731008.364 -11359586.207)


In [4]:
hospital_gdf.head()

,WKT,TOWN,NAME,STATE,ZIPCODE,ADDRESS,SHORTNAME,SHAPE_X,SHAPE_Y,geometry
0,POINT (-71.3418010264452 42.6472178773122),Lowell,Lowell General Hospital,MA,1854,295 Varnum Avenue,Lowell General Hospital,-71.341801,42.647218,POINT (4747466.578 -11520167.009)
1,POINT (-71.061448023266 42.4603867279576),Melrose,Hallmark Health System - Melrose-Wakefield Hos...,MA,2176,585 Lebanon Street,Melrose-Wakefield Hospital,-71.061448,42.460387,POINT (4726668.629 -11423315.492)
2,POINT (-71.528163447604 42.1333878673145),Milford,Milford Regional Medical Center,MA,1757,14 Prospect Street,Milford Regional Medical Center,-71.528163,42.133388,POINT (4690267.283 -11585327.833)
3,POINT (-71.7912737597095 42.2493973682464),Worcester,Saint Vincent Hospital,MA,1608,123 Summer Street,St. Vincent Hospital,-71.791274,42.249397,POINT (4703181.401 -11678411.837)
4,POINT (-71.1052811425066 42.3398506547821),Boston,Beth Israel Deaconess Medical Center - East Ca...,MA,2215,330 Brookline Avenue,Beth Israel Deaconess Med Ctr East,-71.105281,42.339851,POINT (4713250.615 -11438366.700)


In [5]:
school_gdf.head()

,WKT,Description_2,City_Town_Geographic,Master_Address_ID,Name,Fax_No,City_Town_Mailing,Phone_No,State,ESE_Code,Address,Description,Principal,Type,Grades_Offered,ZIP_Code,SHAPE_X,SHAPE_Y,geometry
0,POINT (-70.9318491057449 41.6545680269549),NaN,NEW BEDFORD,4065991,Hayden/McFadden School,508-994-0163,New Bedford,508-997-4511,MA,2010078,361 Cedar Grove,Public,Tammy Morgan,PUB,"PK,K,01,02,03,04,05",2746,-70.931849,41.654568,POINT (4636965.302 -11379009.899)
1,POINT (-72.6451528932961 42.0734436731309),NaN,AGAWAM,2270173,Agawam High School,413-821-0536,Agawam,413-821-0521,MA,50505,760 Cooper Street,Public,Thomas Schnepp,PUB,"09,10,11,12",1001,-72.645153,42.073444,POINT (4683594.326 -11989720.535)
2,POINT (-71.2221815242066 42.0253054269432),NaN,MANSFIELD,2554008,Roland Green School,508-261-7415,Mansfield,508-261-1561,MA,1670003,29 Dean Street,Public,Joanne Jordan,PUB,PK,2048,-71.222182,42.025305,POINT (4678235.601 -11478672.516)
3,POINT (-71.0208840975658 42.4197977682776),NaN,REVERE,3537730,A.C. Whelan Elementary School,781-333-2074,Revere,781-388-7510,MA,2480003,107 Newhall Street,Public,Jamie Flynn,PUB,"K,01,02,03,04,05",2151,-71.020884,42.419798,POINT (4722150.287 -11409416.711)
4,POINT (-71.0366938141637 42.3939387028766),NaN,CHELSEA,3688851,Joseph A Browne School,617-889-8459,Chelsea,617-466-5235,MA,570055,180 Walnut Street,Public,Julie Shea,PUB,"05,06,07,08",2150,-71.036694,42.393939,POINT (4719271.669 -11414830.330)


In [6]:
school_gdf.drop('Description_2', axis=1, inplace=True)
school_gdf.head()

,WKT,City_Town_Geographic,Master_Address_ID,Name,Fax_No,City_Town_Mailing,Phone_No,State,ESE_Code,Address,Description,Principal,Type,Grades_Offered,ZIP_Code,SHAPE_X,SHAPE_Y,geometry
0,POINT (-70.9318491057449 41.6545680269549),NEW BEDFORD,4065991,Hayden/McFadden School,508-994-0163,New Bedford,508-997-4511,MA,2010078,361 Cedar Grove,Public,Tammy Morgan,PUB,"PK,K,01,02,03,04,05",2746,-70.931849,41.654568,POINT (4636965.302 -11379009.899)
1,POINT (-72.6451528932961 42.0734436731309),AGAWAM,2270173,Agawam High School,413-821-0536,Agawam,413-821-0521,MA,50505,760 Cooper Street,Public,Thomas Schnepp,PUB,"09,10,11,12",1001,-72.645153,42.073444,POINT (4683594.326 -11989720.535)
2,POINT (-71.2221815242066 42.0253054269432),MANSFIELD,2554008,Roland Green School,508-261-7415,Mansfield,508-261-1561,MA,1670003,29 Dean Street,Public,Joanne Jordan,PUB,PK,2048,-71.222182,42.025305,POINT (4678235.601 -11478672.516)
3,POINT (-71.0208840975658 42.4197977682776),REVERE,3537730,A.C. Whelan Elementary School,781-333-2074,Revere,781-388-7510,MA,2480003,107 Newhall Street,Public,Jamie Flynn,PUB,"K,01,02,03,04,05",2151,-71.020884,42.419798,POINT (4722150.287 -11409416.711)
4,POINT (-71.0366938141637 42.3939387028766),CHELSEA,3688851,Joseph A Browne School,617-889-8459,Chelsea,617-466-5235,MA,570055,180 Walnut Street,Public,Julie Shea,PUB,"05,06,07,08",2150,-71.036694,42.393939,POINT (4719271.669 -11414830.330)


In [7]:
boston_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 22 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   Unnamed: 0  506 non-null    int64   
 1   TOWN        506 non-null    object  
 2   TOWNNO      506 non-null    int64   
 3   TRACT       506 non-null    int64   
 4   LON         506 non-null    float64 
 5   LAT         506 non-null    float64 
 6   MEDV        506 non-null    float64 
 7   CMEDV       506 non-null    float64 
 8   CRIM        506 non-null    float64 
 9   ZN          506 non-null    float64 
 10  INDUS       506 non-null    float64 
 11  CHAS        506 non-null    int64   
 12  NOX         506 non-null    float64 
 13  RM          506 non-null    float64 
 14  AGE         506 non-null    float64 
 15  DIS         506 non-null    float64 
 16  RAD         506 non-null    int64   
 17  TAX         506 non-null    int64   
 18  PTRATIO     506 non-null    float64 
 19  

In [8]:
hospital_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   WKT        82 non-null     object  
 1   TOWN       82 non-null     object  
 2   NAME       82 non-null     object  
 3   STATE      82 non-null     object  
 4   ZIPCODE    82 non-null     int64   
 5   ADDRESS    82 non-null     object  
 6   SHORTNAME  82 non-null     object  
 7   SHAPE_X    82 non-null     float64 
 8   SHAPE_Y    82 non-null     float64 
 9   geometry   82 non-null     geometry
dtypes: float64(2), geometry(1), int64(1), object(6)
memory usage: 6.5+ KB


In [9]:
school_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1773 entries, 0 to 1772
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   WKT                   1773 non-null   object  
 1   City_Town_Geographic  1773 non-null   object  
 2   Master_Address_ID     1756 non-null   object  
 3   Name                  1773 non-null   object  
 4   Fax_No                1773 non-null   object  
 5   City_Town_Mailing     1773 non-null   object  
 6   Phone_No              1773 non-null   object  
 7   State                 1773 non-null   object  
 8   ESE_Code              1773 non-null   int64   
 9   Address               1773 non-null   object  
 10  Description           1773 non-null   object  
 11  Principal             1773 non-null   object  
 12  Type                  1773 non-null   object  
 13  Grades_Offered        1772 non-null   object  
 14  ZIP_Code              1773 non-null   int64   
 

In [10]:
sorted(boston_gdf.TOWN.unique())

['Arlington',
 'Ashland',
 'Bedford',
 'Belmont',
 'Beverly',
 'Boston Allston-Brighton',
 'Boston Back Bay',
 'Boston Beacon Hill',
 'Boston Charlestown',
 'Boston Dorchester',
 'Boston Downtown',
 'Boston East Boston',
 'Boston Forest Hills',
 'Boston Hyde Park',
 'Boston Mattapan',
 'Boston North End',
 'Boston Roxbury',
 'Boston Savin Hill',
 'Boston South Boston',
 'Boston West Roxbury',
 'Braintree',
 'Brookline',
 'Burlington',
 'Cambridge',
 'Canton',
 'Chelsea',
 'Cohasset',
 'Concord',
 'Danvers',
 'Dedham',
 'Dover',
 'Duxbury',
 'Everett',
 'Framingham',
 'Hamilton',
 'Hanover',
 'Hingham',
 'Holbrook',
 'Hull',
 'Lexington',
 'Lincoln',
 'Lynn',
 'Lynnfield',
 'Malden',
 'Manchester',
 'Marblehead',
 'Marshfield',
 'Medfield',
 'Medford',
 'Melrose',
 'Middleton',
 'Millis',
 'Milton',
 'Nahant',
 'Natick',
 'Needham',
 'Newton',
 'Norfolk',
 'North Reading',
 'Norwell',
 'Norwood',
 'Peabody',
 'Pembroke',
 'Quincy',
 'Randolph',
 'Reading',
 'Revere',
 'Rockland',
 'Sale

### Check that the characteristics of each column are as expected as defined above

In [11]:
assert len(boston_gdf.TOWN.unique()) == len(boston_gdf.TOWNNO.unique()), 'Number of unique town names must be equal to number of unique town numbers'
assert all(boston_gdf.TRACT) >= 0, 'ID numbers must be greater than or equal to zero'
assert -180 <= all(boston_gdf.LON) <= 180, 'Longitude must be between -180 and 180'
assert -90 <= all(boston_gdf.LAT) <= 90, 'Latitude must be between -90 and 90'

### Dropping the MEDV column as CMEDV will be used as our target variable and we do not want data leakage

In [12]:
boston_gdf.drop('MEDV', axis=1, inplace=True)

### Dropping TOWN because it repeats information in TOWNNO

In [13]:
boston_gdf.drop('TOWN', axis=1, inplace=True)

## Feature engineering

In [32]:
# Create a map
m_2 = Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=13)

# Color vector with 92 unique colors
colors = []

for _ in range(boston_gdf['TOWNNO'].nunique()):
    colors.append('#%06X' % randint(0, 0xFFFFFF))

# Add a bubble map to the base map
for i in range(len(boston_gdf)):
#     print(i)
    if i == 8:
#         print('here')
        Circle(
            location=[boston_gdf.iloc[i]['LAT'], boston_gdf.iloc[i]['LON']],
            radius=1000,
            color=colors[boston_gdf.iloc[i]['TOWNNO']],
            popup='HOME '+str(i)+'\n'+str(boston_gdf.to_crs(epsg=4326).geometry.iloc[i])).add_to(m_2)
    else:
        Circle(
            location=[boston_gdf.iloc[i]['LAT'], boston_gdf.iloc[i]['LON']],
            radius=10,
            color=colors[boston_gdf.iloc[i]['TOWNNO']],
            popup='HOME '+str(i)+'\n'+str(boston_gdf.to_crs(epsg=4326).geometry.iloc[i])).add_to(m_2)

# Add a bubble map for hospitals
for i in range(0,len(hospital_gdf)):
    Circle(
        location=[hospital_gdf.iloc[i]['SHAPE_Y'], hospital_gdf.iloc[i]['SHAPE_X']],
        radius=10,
        color='red').add_to(m_2)
    
# Add a bubble map for hospitals
for i in range(0,len(school_gdf)):
    Circle(
        location=[school_gdf.iloc[i]['SHAPE_Y'], school_gdf.iloc[i]['SHAPE_X']],
        radius=10,
        color='black').add_to(m_2)  
    
m_2

In [26]:
boston_gdf.iloc[8,:]

Unnamed: 0                                                                 8
TOWNNO                                                                     3
TRACT                                                                   2043
LON                                                               -70.892442
LAT                                                                 42.52326
CMEDV                                                                   16.5
CRIM                                                                 0.21124
ZN                                                                      12.5
INDUS                                                                   7.87
CHAS                                                                       0
NOX                                                                    0.524
RM                                                                     5.631
AGE                                                                    100.0

In [37]:
boston_gdf['dist_nearest_hospital'] = [np.inf for _ in range(len(boston_gdf))]
for idx, hospital in hospital_gdf.iterrows():
    boston_gdf['dist_nearest_hospital'] = [min(a,b) for a,b in zip(boston_gdf['dist_nearest_hospital'],
                                              boston_gdf.geometry.distance(hospital.geometry))]

# boston_gdf['number_of_schools_within_1km'] = np.zeros(len(boston_gdf))
boston_gdf['dist_nearest_school'] = [np.inf for _ in range(len(boston_gdf))]
for idx, school in school_gdf.iterrows():
    dist_to_school = boston_gdf.geometry.apply(lambda x: school.geometry.distance(x))
    boston_gdf['dist_nearest_school'] = [min(a,b) for a,b in zip(boston_gdf['dist_nearest_school'], dist_to_school)]
#     boston_gdf['number_of_schools_within_1km'] = [a+1 if dist_to_school[idx] <= 1000 else a for idx, a in enumerate(boston_gdf['number_of_schools_within_1km']) ]
#                                               boston_gdf.geometry.distance(school.geometry))]
#     boston_gdf['dist_nearest_school'] = [min(a,b) for a,b in zip(boston_gdf['dist_nearest_school'],
#                                               boston_gdf.geometry.distance(school.geometry))]

In [39]:
boston_gdf.iloc[8,:]

Unnamed: 0                                                                 8
TOWNNO                                                                     3
TRACT                                                                   2043
LON                                                               -70.892442
LAT                                                                 42.52326
CMEDV                                                                   16.5
CRIM                                                                 0.21124
ZN                                                                      12.5
INDUS                                                                   7.87
CHAS                                                                       0
NOX                                                                    0.524
RM                                                                     5.631
AGE                                                                    100.0

In [38]:
one_km_buffers = boston_gdf.geometry.buffer(1000)
number_of_schools_within_1km = list()
for idx, one_km_buffer in enumerate(one_km_buffers):
    school_count = len(school_gdf.loc[school_gdf.geometry.apply(lambda x: one_km_buffer.contains(x))])
    number_of_schools_within_1km.append(school_count)
    if school_count > 1:
        print('\n \n NEW HOUSE \n ROW NUMBER: '+str(idx)+'\n')
        print('# SCHOOLS ', school_count)#len(school_gdf.loc[school_gdf["geometry"].apply(lambda x: one_km_buffer.contains(x))]))
        print(school_gdf.loc[school_gdf["geometry"].apply(lambda x: one_km_buffer.contains(x)), 'geometry'].to_crs(epsg='4326'))
        print(boston_gdf.to_crs(epsg='4326').geometry.iloc[idx])
boston_gdf['number_of_schools_within_1km'] = number_of_schools_within_1km
#     if school_count > 1:
#         print('\n \n NEW HOUSE \n ROW NUMBER: '+str(idx)+'\n')
#         print('# SCHOOLS ', school_count)#len(school_gdf.loc[school_gdf["geometry"].apply(lambda x: one_km_buffer.contains(x))]))
#         print(school_gdf.loc[school_gdf["geometry"].apply(lambda x: one_km_buffer.contains(x)), 'geometry'].to_crs(epsg='4326'))
#         print(boston_gdf.to_crs(epsg='4326').geometry.iloc[idx])


 
 NEW HOUSE 
 ROW NUMBER: 8

# SCHOOLS  2
1529    POINT (42.52206 -70.89324)
1633    POINT (42.52201 -70.89338)
Name: geometry, dtype: geometry
POINT (42.52325953846153 -70.89244248131541)

 
 NEW HOUSE 
 ROW NUMBER: 22

# SCHOOLS  2
515     POINT (42.46108 -70.96266)
1217    POINT (42.46923 -70.96662)
Name: geometry, dtype: geometry
POINT (42.46695138461538 -70.96396109865472)

 
 NEW HOUSE 
 ROW NUMBER: 23

# SCHOOLS  2
516     POINT (42.46506 -70.95462)
1334    POINT (42.46424 -70.95490)
Name: geometry, dtype: geometry
POINT (42.46695138461538 -70.95672887892377)

 
 NEW HOUSE 
 ROW NUMBER: 27

# SCHOOLS  2
175     POINT (42.47159 -70.93663)
1447    POINT (42.47365 -70.93308)
Name: geometry, dtype: geometry
POINT (42.47292346153845 -70.93535365171898)

 
 NEW HOUSE 
 ROW NUMBER: 28

# SCHOOLS  2
592     POINT (42.46610 -70.93097)
1447    POINT (42.47365 -70.93308)
Name: geometry, dtype: geometry
POINT (42.46865769230769 -70.93101431988042)

 
 NEW HOUSE 
 ROW NUMBER: 32

# SCHOOLS

POINT (42.37566392307693 -71.03146181614349)

 
 NEW HOUSE 
 ROW NUMBER: 391

# SCHOOLS  2
1697    POINT (42.38854 -71.00417)
1731    POINT (42.39162 -71.00505)
Name: geometry, dtype: geometry
POINT (42.38846123076922 -71.00574725710015)

 
 NEW HOUSE 
 ROW NUMBER: 392

# SCHOOLS  3
1692    POINT (42.36968 -71.03412)
1698    POINT (42.36925 -71.03422)
1729    POINT (42.36556 -71.03489)
Name: geometry, dtype: geometry
POINT (42.36713238461539 -71.0322653961136)

 
 NEW HOUSE 
 ROW NUMBER: 400

# SCHOOLS  2
1668    POINT (42.33132 -71.05054)
1732    POINT (42.33803 -71.05215)
Name: geometry, dtype: geometry
POINT (42.33403001538461 -71.05251561136025)

 
 NEW HOUSE 
 ROW NUMBER: 407

# SCHOOLS  2
1737    POINT (42.34841 -71.06905)
1763    POINT (42.34869 -71.06812)
Name: geometry, dtype: geometry
POINT (42.34750984615385 -71.06874792675634)

 
 NEW HOUSE 
 ROW NUMBER: 408

# SCHOOLS  2
1690    POINT (42.34075 -71.07242)
1737    POINT (42.34841 -71.06905)
Name: geometry, dtype: geometry
P

POINT (42.38538987692309 -70.9776219581465)

 
 NEW HOUSE 
 ROW NUMBER: 504

# SCHOOLS  2
998     POINT (42.38037 -70.97946)
1584    POINT (42.38023 -70.97925)
Name: geometry, dtype: geometry
POINT (42.37310446153845 -70.9800326980568)


In [17]:
boston_gdf.describe()

,Unnamed: 0,TOWNNO,TRACT,LON,LAT,CMEDV,CRIM,ZN,INDUS,CHAS,...,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,dist_nearest_hospital,dist_nearest_school,number_of_schools_within_1km
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,...,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,252.500000,47.531621,2700.357708,-71.090748,42.360205,22.528854,3.613524,11.363636,11.136779,0.069170,...,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,6805.115348,1344.146452,0.675889
std,146.213884,27.571401,1380.038110,0.121188,0.105411,9.182176,8.601545,23.322453,6.860353,0.253994,...,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,6003.072155,1002.722703,0.942764
min,0.000000,0.000000,1.000000,-71.465395,42.042081,5.000000,0.006320,0.000000,0.460000,0.000000,...,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,186.688086,81.877037,0.000000
25%,126.250000,26.250000,1303.250000,-71.149950,42.299349,17.025000,0.082045,0.000000,5.190000,0.000000,...,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,3278.069646,633.090475,0.000000
50%,252.500000,42.000000,3393.500000,-71.085141,42.363037,21.200000,0.256510,0.000000,9.690000,0.000000,...,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,5150.618727,1130.703227,0.000000
75%,378.750000,78.000000,3739.750000,-71.031663,42.421308,25.000000,3.677083,12.500000,18.100000,0.000000,...,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,8668.550945,1724.896342,1.000000
max,505.000000,91.000000,5082.000000,-70.694762,42.640995,50.000000,88.976200,100.000000,27.740000,1.000000,...,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50928.168752,7292.403310,5.000000


### Seperate explanatory variables from target and split into train and test set (80:20), will be using cross-validation on training set

In [ ]:
X = boston_gdf.copy()
y = X.pop('CMEDV')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2021)

### Calculate a mutual information score to understand how knowledge of one quantity reduces uncertainty about the target variable

* 0 means that the quantities are independent
* The higher the score the better
* Scores > 2 are uncommon as mutual information score increases slowly as it is a logarithmic quantity

In [ ]:
mi_scores = mutual_info_regression(X.loc[:, X.columns != 'geometry'], y, random_state=2021)
mi_scores_series = pd.Series(mi_scores, name="MI Scores", index=X.loc[:, X.columns != 'geometry'].columns).sort_values(ascending=False)

In [ ]:
mi_scores_series

In [ ]:
mi_scores_series.plot.barh()
plt.xlabel('Mutual Information Score')
plt.ylabel('Feature')
plt.show()

### Correlation coefficients

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(15,15))
cm = np.corrcoef(X_train.loc[:, X.columns != 'geometry'].values.T)
sns.set(font_scale=1.5)
hm = sns.heatmap(cm,
                cbar=True,
                annot=True,
                square=True,
                fmt='.2f',
                annot_kws={'size': 15},
                yticklabels=X_train.loc[:, X.columns != 'geometry'].columns,
                xticklabels=X_train.loc[:, X.columns != 'geometry'].columns,
                ax=ax)

### Normalizing data using StandardScaler()

In [ ]:
x_scaler = StandardScaler()
X_train_sc = x_scaler.fit_transform(X_train.loc[:, X.columns != 'geometry'])
X_test_sc = x_scaler.transform(X_test.loc[:, X.columns != 'geometry'])

### Principle Component Analysis (PCA)

In [ ]:
pca = PCA()
X_train_pca = pca.fit_transform(X_train_sc)

component_names = [f"PC{i+1}" for i in range(X_train_pca.shape[1])]
X_train_pca_df = pd.DataFrame(X_train_pca, columns=component_names)

X_train_pca_df.head()

In [ ]:
loadings = pd.DataFrame(
    pca.components_.T,  # transpose the matrix of loadings
    columns=component_names,  # so the columns are the principal components
    index=X_train.loc[:, X.columns != 'geometry'].columns,  # and the rows are the original features
)
loadings = pd.concat([loadings, pd.DataFrame(X_train.loc[:, X.columns != 'geometry'].columns, columns=['col_names'], index=X_train.loc[:, X.columns != 'geometry'].columns)], axis=1)

In [ ]:
loadings.head()

In [ ]:
plt.figure(figsize=(20,10))
plt.bar(component_names, pca.explained_variance_ratio_)

In [ ]:
%matplotlib inline

fg = sns.FacetGrid(data=loadings, hue='col_names', hue_order=X_train.loc[:, X.columns != 'geometry'].columns, height=8)
fg.map(plt.scatter, 'PC1', 'PC2').add_legend(ncol=1)
for x, y, label in zip(loadings.PC1, loadings.PC2, loadings.col_names):
    plt.text(x=x+0.01, y=y+0.01, s=label)

### Benchmark model: Linear Regression

In [ ]:
def mean_absolute_percentage_error(true, predicted):
    return np.mean(abs((true-predicted)/true))*100

In [ ]:
benchmark_model = LinearRegression(n_jobs=-1)
benchmark_model.fit(X_train_sc, y_train)
train_pred = benchmark_model.predict(X_train_sc)
train_RMSE = mean_squared_error(y_train, train_pred, squared=False)
train_MAPE = mean_absolute_percentage_error(y_train, train_pred)
test_pred = benchmark_model.predict(X_test_sc)
test_RMSE = mean_squared_error(y_test, test_pred, squared=False)
test_MAPE = mean_absolute_percentage_error(y_test, test_pred)

In [ ]:
benchmark_res = {'train_RMSE':[train_RMSE], 'test_RMSE':[test_RMSE], 'train_MAPE':[train_MAPE], 'test_MAPE':[test_MAPE]}
evaluation = pd.DataFrame(benchmark_res, index=['benchmark'])

In [ ]:
evaluation

In [ ]:
# can try decision tree regressor